In [10]:
import pandas as pd
import numpy as np
import pymc as pm
import matplotlib.pyplot as plt
from scipy.stats import linregress
from tqdm import tqdm

In [11]:
# Load the data
data = pd.read_csv('component_month_counts.csv')
data['Month'] = pd.to_datetime(data['Month'])

# Define the target end date for all components
END_DATE = pd.to_datetime("2024-11-01")

# Define the range of months for each component individually
components = data['Component'].unique()
expanded_data = pd.DataFrame()
last_observed_date = []
OFFSET = 0.01
decay_rates = []

for component in components:
    # Filter data for the current component
    component_data = data[data['Component'] == component]
    last_observed_date.append(component_data['Month'].max())

    # Define the full range of months for this component
    all_months = pd.date_range(start=component_data['Month'].min(), end=END_DATE, freq='MS')

    # Create a DataFrame with all months for this component
    component_months = pd.DataFrame({'Component': component, 'Month': all_months})

    # Merge to include counts, filling missing months with zero counts
    component_months = component_months.merge(component_data, on=['Component', 'Month'], how='left').fillna(0)
    component_months['Count'] = component_months['Count'].astype(int)

    component_months['Count_with_offset'] = component_months['Count'] + OFFSET

    # Append to the main expanded data
    expanded_data = pd.concat([expanded_data, component_months], ignore_index=True)

    time = (component_months['Month'] - component_data['Month'].min()).dt.days // 30
    log_counts = np.log(component_months['Count_with_offset'])

    slope, _, _, _, _ = linregress(time, log_counts)
    decay_rate = -slope

    if decay_rate > 0:
      decay_rates.append(decay_rate)

In [12]:
traces = []
DECAY_RATE = np.mean(decay_rates)

for component in tqdm(components):
  # Select data for the specific component
  component_data = expanded_data[expanded_data['Component'] == component]
  component_counts = component_data['Count'].values
  time_since_observe = (component_data['Month'] - component_data['Month'].min()).dt.days // 30

  # PyMC Model
  with pm.Model() as model:
    # Priors for alpha and beta using Exponential distributions
    alpha = pm.Exponential("alpha", 1.0)
    beta = pm.Exponential("beta", 1.0)

    # Time-decaying lambda using alpha, beta, and learned decay_rate
    lambda_ = pm.Gamma("lambda", alpha, beta)
    lambda_t = lambda_ * np.exp(-DECAY_RATE * time_since_observe)

    # Likelihood for observed counts
    counts_observed = pm.Poisson("counts_observed", mu=lambda_t, observed=component_counts)

    # Run the NUTS sampler
    trace = pm.sample(3000, tune=1000, target_accept=0.9, return_inferencedata=True)

    traces.append(trace)

  # # Plot trace to check convergence
  # pm.plot_trace(trace)
  # plt.show()

  # # Summary of the posterior
  # summary = pm.summary(trace)
  # print(summary)

  0%|          | 0/388 [00:00<?, ?it/s]Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, beta, lambda]


/Users/alanfong/Desktop/UCI/capstone_mcmc/.venv/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Sampling 4 chains for 1_000 tune and 3_000 draw iterations (4_000 + 12_000 draws total) took 2 seconds.
  0%|          | 1/388 [00:06<41:40,  6.46s/it]Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, beta, lambda]


/Users/alanfong/Desktop/UCI/capstone_mcmc/.venv/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Sampling 4 chains for 1_000 tune and 3_000 draw iterations (4_000 + 12_000 draws total) took 2 seconds.
  1%|          | 2/388 [00:10<30:42,  4.77s/it]Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, beta, lambda]


...

Sampling 4 chains for 1_000 tune and 3_000 draw iterations (4_000 + 12_000 draws total) took 2 seconds.
 99%|█████████▉| 386/388 [25:43<00:07,  3.89s/it]Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, beta, lambda]


/Users/alanfong/Desktop/UCI/capstone_mcmc/.venv/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Sampling 4 chains for 1_000 tune and 3_000 draw iterations (4_000 + 12_000 draws total) took 2 seconds.
100%|█████████▉| 387/388 [25:46<00:03,  3.73s/it]Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, beta, lambda]


/Users/alanfong/Desktop/UCI/capstone_mcmc/.venv/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Sampling 4 chains for 1_000 tune and 3_000 draw iterations (4_000 + 12_000 draws total) took 2 seconds.
100%|██████████| 388/388 [25:50<00:00,  4.00s/it]


In [13]:
# Define the number of prediction samples and the future time point
num_prediction_samples = 1000
PREDICTION_DATE = pd.to_datetime("2024-12-01") # Predicting 1 month into the future
results = []

for i in tqdm(range(len(traces))):
  future_time = (PREDICTION_DATE - last_observed_date[i]).days // 30

  # Extract posterior samples for each parameter
  # decay_rate_samples = traces[i].posterior["decay_rate"].values.flatten()
  # decay_rate = prior_decay_rates["Windows SMB"]
  # alpha_samples = traces[i].posterior["alpha"].values.flatten()
  # beta_samples = traces[i].posterior["beta"].values.flatten()
  lambda_samples = traces[i].posterior["lambda"].values.flatten()

  # Generate predictions for the next month
  predicted_counts = []

  for n in range(num_prediction_samples):
    # Draw samples for decay_rate and lambda from the posterior
    # decay_rate = decay_rate_samples[n]
    lambda_ = lambda_samples[n]

    # Calculate the decayed rate for the future time point
    lambda_future = lambda_ * np.exp(-DECAY_RATE * future_time)

    # Draw a predicted count from the Poisson distribution with the decayed rate
    predicted_count = np.random.poisson(lambda_future)
    predicted_counts.append(predicted_count)

  # # Calculate summary statistics for the predictions
  predicted_mean = np.mean(predicted_counts)
  predicted_std = np.std(predicted_counts)
  # predicted_95_ci = np.percentile(predicted_counts, [2.5, 97.5])

  # print(f"Predicted Mean Count for Next Month: {predicted_mean}")
  # print(f"Predicted Standard Deviation: {predicted_std}")
  # print(f"95% Prediction Interval: {predicted_95_ci}")

  results.append([components[i], 
                  float(trace.posterior['alpha'].mean().values),
                  float(trace.posterior['beta'].mean().values), 
                  float(trace.posterior['lambda'].mean().values), 
                  predicted_mean, 
                  predicted_std])

100%|██████████| 388/388 [00:00<00:00, 707.20it/s]


In [14]:
output = pd.DataFrame(results, columns=['Component', 'Mean_Alpha', 'Mean_Beta', 'Mean_Lambda', 'Predicted_Mean', 'Predicted_Std'])
output.to_csv('output.csv', index=False)